 <b><p style="text-align:center;">
    <font size ="8" color ="Green">
        Gangue (Silica Waste) Forecast in Flotation Concentrate
    </font>
</b>

Mined ores are mostly mixtures of extractable minerals and nonvaluable material (gangue). Mineral processing (a.k.a. ore dressing, ore beneficiation) follows mining and prepares the ore for extraction of the valuable metal. A principal step in mineral processing is physical separation of the particles of valuable minerals from the gangue, to produce an enriched portion (concentrate) containing most of the valuable minerals, and a discard (tailing) containing predominantly the gangue.

A separation of minerals by exploiting difference of surface properties (hydrophobicity) is called flotation. The reverse cationic flotation is commonly used to separate iron from silica. By adjusting the 'chemistry' of the pulp by adding various chemical reagents, iron minerals remain in the water and create sediment with a high concentration of iron (valuable minerals). At the same time, silica particles (gangue) attach to air bubbles and float to the surface.

<p style="text-align:center;">
    <img width="500" alt="Reverse cationic flotation of iron ore" src="https:\\github.com\ginsaputra\gangue-forecast-in-flotation-concentrate\blob\main\reverse-cationic-flotation-iron-silica.png?raw=true">

Flotation concentrate is periodically sampled to determine its purity (i.e., *%valuable*, *%gangue*). Higher *%gangue* in the concentrate is undesirable as it indicates that most valuable minerals had gone into the tailings. Purity measurement is usually done in a lab and can take some time before process engineers can make any adjustments based on the results. A timely investigation of concentrate purity is, therefore, a fundamental aspect for the control and optimization of the flotation process.

This notebook explores the application of deep learning to forecast **gangue (%silica)** in the flotation concentrate. The forecast will help process engineers assess purity of flotation concentrate and take corrective actions in advance. More specifically, the goal is to tackle the following tasks:
- How many steps (hours) ahead can *%silica in concentrate* be forecasted?
- Is it possible to forecast *%silica in concentrate* without using the data of *%iron in concentrate*?

In [2]:
# Load relevant libraries

%pip install numpy==1.19.5

%pip install libhdf5
%pip install h5py
%pip install tensorflow

import os, datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow 
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, Callback
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Plotly libraries
import chart_studio.plotly as py
import plotly.graph_objects as go
import plotly.express as px

# Offline mode
import plotly.offline as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

from IPython.display import HTML, display

# Formatting options
pd.options.display.float_format = '{:,.2f}'.format

# Turn on execution time for JupyterLab
try:
    %reload_ext autotime
except:
    %pip install ipython-autotime
    %load_ext autotime

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement libhdf5 (from versions: none)
ERROR: No matching distribution found for libhdf5


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ImportError: cannot import name 'export_saved_model' from 'tensorflow.python.keras.saving.saved_model' (C:\Users\pdudar\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\keras\saving\saved_model\__init__.py)

# Flotation Data

The dataset was obtained from a mineral processing plant separating silica from iron ore using the reverse cationic flotation method. Continuous process data were collected from 1 a.m. on March 10, 2017 to 11 p.m. on September 9, 2017.

Each row of data consists of 23 measurements that can be categorized into four types:
- raw materials (column 2-3);
- environment variables (column 4-8);
- process variables (column 9-22);
- processed materials (column 23-24).

Raw materials and processed materials were sampled on an hourly basis while the others were sampled every 20 second.

In [ ]:
file = r'C:/Users/pdudar/anaconda3/git/MiningProcessData/MiningProcess_Flotation_Plant_Database.csv'
cols_renamed = [
    'date',          # Timestamp of measurements, formatted YYYY-MM-DD HH:MM:SS
    'feed_iron',     # %Iron (valuables) in the ore being fed into the flotation cell
    'feed_silica',   # %Silica (gangue) in the ore being fed into the cell
    'starch_flow',   # Amount of starch (reagent) added into the cell, measured in m^3\h
    'amina_flow',    # Amount of amina (reagent) added into the cell, measured in m^3\h
    'pulp_flow',     # Amount of ore pulp fed into the cell, measured in tonnes\hour
    'pulp_ph',       # Acidity\alkalinity of ore pulp on a scale of 0-14
    'pulp_density',  # Amount of ore in the pulp, between 1-3 kg\cm^3
    'air_col1',      # Volume of air injected into the cell, measured in Nm3\h
    'air_col2',
    'air_col3',
    'air_col4',
    'air_col5',
    'air_col6',
    'air_col7',
    'level_col1',    # Froth height in the cell, measured in mm
    'level_col2',
    'level_col3',
    'level_col4',
    'level_col5',
    'level_col6',
    'level_col7',
    'conc_iron',     # Lab measurement: %Iron in the end of flotation process
    'conc_silica']   # Lab measurement: %Silica in the end of flotation process

df = pd.read_csv(
                file,
                header=0,
                names=cols_renamed,
                parse_dates=['date'],
                infer_datetime_format=True,
                decimal=',')
df.head()

In [ ]:
float_cols = df.columns[1:]

for col in float_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce', downcast='float')

df.info()

In [ ]:
df['feed_iron_title'] = 'Iron Feed'
df['feed_silica_title'] = 'Silica Feed'
df['conc_iron_title'] = 'Iron Concentration'
df['conc_silica_title'] = 'Silica Concentration'

# Preprocessing Time-Series Data

Column `date` is resampled to reduce the frequency of our time-series data into an **hourly basis**. This is achieved by selecting only the first measurements of each hour.

In [ ]:
# Resample data to 15 min basis
df = df.set_index('date').resample('H').first()
df.shape

Upon inspecting the data, 318 rows containing missing values are found between `2017-03-16 06:00` to `2017-03-29 11:00`. Missing values introduce discontinuity in the time-series data and can be detrimental to our forecast. Therefore, only the data starting from `2017-03-29 12:00` will be further used.

In [ ]:
nans = df[df.isna().any(axis=1)]  # Check for missing values
print(f'Total rows with NaNs: {nans.shape[0]}\n')
nans

In [ ]:
# Remove data with time discontinuity
df = df['2017-03-29 12:00:00':]
df

When sampling on 15 minute intervals there are clear gaps in data on an hourly interval.

We can try to fill these gaps by interpolating between the values. This is not a guarantee of what is actually happening, but we'll 
see what happens

In [ ]:
df = df.interpolate(axis=0, method='linear')
df.head(30)

The following plot shows mineral content before (i.e., in the feed) and after flotation process (in the concentrate). As can be observed from the figure, the purpose of flotation is to increase recovery of iron mineral while reducing the gangue (silica).

During some periods (e.g., May 13 to June 13), mineral content in the feed was constant but the resulting content in the concentrate fluctuated. This suggests that *%iron* and *%silica in concentrate* are not solely governed by the content of raw materials but other parameters as well (i.e., environment, process variables).

In [ ]:
content = ['feed_iron', 'feed_silica', 'conc_iron', 'conc_silica']
palette = ['#FB6542', '#FFBB00', '#3F681C', '#375E97']

# Plot mineral content before and after flotation
plt.style.use('ggplot')
fig, ax = plt.subplots(figsize=(18,6))
for pct, color in zip(content, palette):
    ax.plot(df.index.values, pct, data=df, color=color)
ax.set_title('Mineral content in feed and concentrate',
             loc='left', weight='bold', size=16)
ax.set_ylabel('% Mineral')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.legend(loc='center left')
plt.show()

In [ ]:
# Create figure with secondary y-axis
from plotly.subplots import make_subplots
subfig = make_subplots(specs=[[{"secondary_y": True}]], shared_yaxes=True)

# Generate a Plotly TimeSeries chart for each monitoring station

fig = px.line(df, 
              x = df.index.values,
              y = 'feed_iron',
              title = 'Mineral content in feed and concentrate',
              hover_name = 'feed_iron_title', 
              hover_data = ['feed_iron'],
               render_mode="webgl")
             

fig2 = px.line(df, 
              x = df.index.values,
              y = 'feed_silica',
              hover_name = 'feed_silica_title', 
              hover_data = ['feed_silica'],
               render_mode="webgl")


fig3 = px.line(df, 
              x = df.index.values,
              y = 'conc_iron',
              hover_name = 'conc_iron_title', 
              hover_data = ['conc_iron'],
               render_mode="webgl")
             

fig4 = px.line(df, 
              x = df.index.values,
              y = 'conc_silica',
              hover_name = 'conc_silica_title', 
              hover_data = ['conc_silica'],
               render_mode="webgl")

fig.update_traces(yaxis="y1")
fig2.update_traces(yaxis="y1")
fig3.update_traces(yaxis="y1")
fig4.update_traces(yaxis="y1")

subfig.add_traces(fig.data + fig2.data + fig3.data + fig4.data)

subfig.layout.xaxis.title="DATE"
subfig.layout.yaxis.title="MINERAL CONCENTRATION (%)"
subfig.layout.yaxis.type="linear"

#subfig.layout.yaxis2.type="linear"
#subfig.layout.yaxis2.title="UPPER AND LOWER GUIDELINE RECCOMENDATIONS"

# recoloring is necessary otherwise lines from fig und fig2 would share each color
# e.g. Linear-, Log- = blue; Linear+, Log+ = red... we don't want this
subfig.for_each_trace(lambda t: t.update(line=dict(color=t.marker.color))) 

# Add figure title
subfig.update_layout(title_text = 'Mineral content in feed and concentrate: Time Series with Range Slider')

subfig.update_layout(yaxis=dict(title='(%) MINERAL CONCENTRATION', tick0=0, anchor='x', rangemode='normal'))
#subfig.update_layout(yaxis2=dict(title='UPPER AND LOWER GUIDELINE RECCOMENDATIONS',
                                # tick0=0, side='right', anchor='x', rangemode='normal'))

# Set y-axes titles
subfig.update_yaxes(title_text="<b>CONCENTRATION (%)</b>", secondary_y=False, autorange=True)

subfig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1DAY", step="day", stepmode="backward"),
            dict(count=6, label="6MNTH", step="month", stepmode="backward"),
            dict(count=1, label="1MNTH", step="month", stepmode="backward"),
            dict(count=6, label="6MNTH", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1YR", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

# Chart options
subfig.update_layout(plot_bgcolor = "RGB(45,45,48)")  #HEX 2d2d30 
subfig.update_layout(paper_bgcolor = "RGB(37,37,38)") #HEX 252526

subfig.update_layout(
    font_color="RGB(131,148,150)",  	#839496
    title_font_color="RGB(131,148,150)",  	#839496
    legend_title_font_color="RGB(131,148,150)"
    ) #839496

subfig.update_layout(height=600, width=1200)

# Append wq station names to .html files
subfig.write_html("Mineral content in feed and concentrate: Time Series with Range Slider.html", include_plotlyjs='cdn')
subfig.show()

The column `conc_iron` is dropped from the dataframe because we want to look at forecasting **%silica in concentrate** without including **iron in concentrate** as a feature. The data are then normalized as they have different units and scales.

In [ ]:
from sklearn.preprocessing import RobustScaler

cols = ['conc_silica', 'feed_iron', 'feed_silica',
        'starch_flow', 'amina_flow', 'pulp_flow',
        'pulp_ph', 'pulp_density', 'air_col1',
        'air_col2', 'air_col3', 'air_col4',
        'air_col5', 'air_col6', 'air_col7',
        'level_col1', 'level_col2', 'level_col3',
        'level_col4', 'level_col5', 'level_col6', 
        'level_col7', 'conc_iron']

cols.insert(0, cols.pop(         # Moving target `conc_silica` to the front
    cols.index('conc_silica')))  # Not necessary but I prefer to do so
df = df.loc[:, cols]
df.to_csv('Flotation_Dataset_by_Hour.csv')  # For safekeeping

# Drop `conc_iron` then normalize all data
values = df.drop('conc_iron', axis=1).values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
scaled[0]  # Show first element of the array

The preprocessing stage involves framing the dataset as a supervised learning problem where we forecast the **%silica in concentrate** 
at the current hour (**t**) given the parameters (i.e., raw materials, environment, process) in prior time steps **(t-n)**). 

We transform the dataset using the `series_to_supervised()` function below, which was adapted from the blog [Machine Learning Mastery](https:\\machinelearningmastery.com\convert-time-series-supervised-learning-problem-python\) by Jason Brownlee.

In [ ]:
# Convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, drop_nan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        drop_nan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    
    for i in range(n_in, 0, -1):   # Input sequence (t-n, ... t-1)
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
        
    for i in range(0, n_out):      # Forecast sequence (t, t+1, ... t+n)
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
            
    agg = pd.concat(cols, axis=1)  # Put it all together
    agg.columns = names
    if drop_nan:                    # Drop rows with NaN values
        agg.dropna(inplace=True)
        
    # Drop columns we don't want to predict
    drop_cols = ['var'+ str(i) +'(t)' for i in range(2,23)]
    agg.drop(columns=drop_cols, axis=1, inplace=True)   
    return agg

In [ ]:
reframed = series_to_supervised(scaled, n_in=1, n_out=1)
reframed  # Show reframed dataset 

# t-1 = 0.25 hrs or 15 mins

Column `var1(t-1)` until `var22(t-1)` are our features and inputs. Measured values of the inputs are lagged 1 hour before the target `var1(t)` (*%silica in concentrate* at time *t*).

Transformed data is further split into three sets: training (60%), validation (20%), and testing (20%). Flotation data in the period between `2017-03-29 12:00` and `2017-08-08 00:00` is used for training\validation purpose, totaling 3157 hours.

The following is an autoregression model to predict the %silica concentrate (var1(t)) using measured values lagged 30 minutes (var1(t-0.5)) as features/inputs

In [ ]:
n_features = 22             # Number of inputs for forecast
n_hours = 1             # Number of hours with which to lag features (1 = 1 hour, 0.5 = 30 mins, 0.25 = 15 mins etc)
n_obs = n_hours*n_features

# Define row size of each split
n_train = int(np.round(len(reframed) * .60))
n_valid = int(np.round(len(reframed) * .20))
n_test = int(np.round(len(reframed) * .20))

# Split dataset by row size
values = reframed.values
train = values[:n_train, :]
valid = values[n_train:(n_train + n_valid), :]
test = values[(n_train + n_valid):, :]

# Each set further split into inputs\features (X) and output (y)
train_X, train_y = train[:, :n_obs], train[:, -1]
valid_X, valid_y = valid[:, :n_obs], valid[:, -1]
test_X, test_y = test[:, :n_obs], test[:, -1]

# Reshape inputs (X) to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
valid_X = valid_X.reshape((valid_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))

print(  # Show the final shape of each set
    f'Training set: {train_X.shape}, {train_y.shape}',
    f'\nValidation set: {valid_X.shape}, {valid_y.shape}',
    f'\nTesting set: {test_X.shape}, {test_y.shape}')

# Building and Training Model

Forecasting gangue in flotation concentrate is a time-series related problem. A variation of the recurrent neural networks (RNN), called the long short-term memory (LSTM), is a deep learning approach that can be implemented to solve the problem.
> *LSTMs have an edge over conventional feed-forward neural networks and RNN in many ways. This is because of their property of selectively remembering patterns for long durations of time.* -[Analytics Vidhya](https:\\www.analyticsvidhya.com\blog\2017\12\fundamentals-of-deep-learning-introduction-to-lstm\)

Model construction and training is done with [Keras](https:\\tensorflow.org\api_docs\python\tf\keras), a Python deep learning library. The model is made up of two [LSTM](https:\\www.tensorflow.org\api_docs\python\tf\keras\layers\LSTM) layers, each with 16 memory cells, and followed by a fully-connected ([Dense](https:\\www.tensorflow.org\api_docs\python\tf\keras\layers\Dense)) layer. In training, the data are grouped into mini batches of 16 training samples. The training process is run for 100 epochs (one epoch is the number of passes needed to complete the entire training samples).

In [ ]:
model = Sequential([   # Define a sequential model
    LSTM(units=16,
         return_sequences=True,
         input_shape=(train_X.shape[1],
                      train_X.shape[2])),
    LSTM(units=16),
    Dense(1)
])
model.compile(
    loss='mae',        # Mean absolute error
    optimizer='adam')  # Learning rate = 0.001
model.summary()        # Display model's architecture

## Fit the model (Sequential)

In [ ]:
os.chdir(r'C:/Users/pdudar/anaconda3/git/MiningProcessData/')

# Load tensorboard notebook extension
%reload_ext tensorboard

# Add TensorBoard callback variables
logdir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tensorflow.keras.callbacks.TensorBoard(log_dir=logdir, 
                                                              histogram_freq=1,
                                                              profile_batch = 100000000)

model_checkpoint = ModelCheckpoint('LSTM_Flotation_Gangue.hdf5/saved_model.pb')

history = model.fit(   # Fit on training data
            train_X,
            train_y,
            epochs=30,
            batch_size=16,
            validation_data=(  # Supply validation data
                valid_X, #X_test
                valid_y), #y_test
            verbose=2,
            shuffle=False,
            callbacks=([model_checkpoint])
        )

## Extract losses from the training history and display

In [ ]:
# Extract losses from training history
train_loss = history.history['loss']
valid_loss = history.history['val_loss']

# Plot learning curves
plt.style.use('ggplot')
fig, ax = plt.subplots(figsize=(9,6))
ax.plot(train_loss, color=palette[0], label='Training')
ax.plot(valid_loss, color=palette[2], label='Validation')
ax.set_title('Learning Curves', loc='left', weight='bold', size=16)
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss (Mean Absolute Error)')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.legend(loc='center right')
plt.show()

The plot above indicates that the model exhibits a good fit where training and validation losses decrease to a point of stability and there is minimal gap between the two values.

In [ ]:
print(f'Best training loss = {min(train_loss):.4f}',
      f'at epoch {train_loss.index(min(train_loss))}',
      f'\nBest validation loss = {min(valid_loss):.4f}',
      f'at epoch {valid_loss.index(min(valid_loss))}')

# Forecasting with LSTM

Forecasting is performed using the trained model and on the data that was not included during training\validation process. Forecasts and actual values are inverted back into their original scales before calculating error scores for the model. Two error metrics are considered in the forecast evaluation: (1) mean absolute error, and (2) root mean squared error.

In [ ]:
# Make prediction using test features
yhat = model.predict(test_X)

# Reshape test data
test_X = test_X.reshape((test_X.shape[0], n_hours*n_features))
test_y = test_y.reshape((len(test_y), 1))

# Invert scaling for forecasts
inv_yhat = np.concatenate((yhat, test_X[:, -(n_features-1):]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]

# Invert scaling for actual values
inv_y = np.concatenate((test_y, test_X[:, -(n_features-1):]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]

# Calculate error scores
mae = mean_absolute_error(inv_y, inv_yhat)
rmse = np.sqrt(mean_squared_error(inv_y, inv_yhat))
print(32*'-'+'\nFORECAST EVALUATION' + '\n' + 32 * '-',
      f'\nMean absolute error    : {mae:.4f}',
      f'\nRoot mean squared error: {rmse:.4f}')

In [ ]:
from scipy.stats import zscore

# Calc z-score for inv_y (% Silica Concentrate) Actual Values
z = zscore(inv_y)
print(z)


In [ ]:

# Define date as x-axis
test_date = df.index[-test_y.shape[0]:]

# Define the 95% Confidence interval (Z-value 1.96)
ci = z * (np.std(inv_y) / np.mean(inv_y))

# Plot actual values and forecasts
plt.style.use('ggplot')
fig, ax = plt.subplots(figsize=(18,6))
ax.plot(test_date, inv_y, color=palette[1], label='Actual Value')
ax.plot(test_date, inv_yhat, color=palette[2], label='Forecast')
ax.fill_between(test_date, (inv_y-ci), (inv_y+ci), color=palette[3],
                alpha=.1, label='95(%) Confidence Interval')
ax.set_title('(%) Silica in Concentrate: Actual Values and Forecasts by LSTM',
             loc='left', weight='bold', size=16)
ax.set_ylabel('Silica in Concentrate (%)')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.legend()
plt.show()

In [ ]:
from plotly.subplots import make_subplots
subfig = make_subplots(specs=[[{"secondary_y": True}]], shared_yaxes=True)

fig = go.Figure([
    go.Scatter(
        name='(%) Silica in Conc: Actual Value',
        x=test_date,
        y=inv_y,
        mode='lines',
        line=dict(color='rgb(24, 48, 96)')
    ),
    
    go.Scatter(
        name='(%) Silica in Conc: Forecast Value',
        x=test_date,
        y=inv_yhat,
        mode='lines',
        line=dict(color='rgb(48, 96, 192)')
    ),
    
    go.Scatter(
        name='95 perc CI: Upper Bound',
        x=test_date,
        y=inv_y+ci,
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=True
    ),
    
    go.Scatter(
        name='95 perc CI: Lower Bound',
        x=test_date,
        y=inv_y-ci,
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=True
    )

])

fig.update_layout(
    yaxis_title='Silica in Conc (%)',
    title='(%) Silica in Conc: Actual Values and Forecasts by LSTM',
    hovermode="x",
    height=500, width=1500
)

subfig.add_traces(fig.data)

subfig.layout.xaxis.title="Date"
subfig.layout.yaxis.title="Silica in Concentrate (%)"
subfig.layout.yaxis.type="linear"

# recoloring is necessary otherwise lines from fig und fig2 would share each color
# e.g. Linear-, Log- = blue; Linear+, Log+ = red... we don't want this
subfig.for_each_trace(lambda t: t.update(line=dict(color=t.marker.color))) 

# Add figure title
subfig.update_layout(title_text = '(%) Silica in Concentrate: Actual Values and Forecasts by LSTM: Time Series with Range Slider')
subfig.update_layout(yaxis=dict(title='Silica in Concentrate (%)', tick0=0, anchor='x', rangemode='normal'))

# Set y-axes titles
subfig.update_yaxes(title_text="<b>Silica in Concentrate (%)</b>", secondary_y=False, autorange=True)

subfig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1DAY", step="day", stepmode="backward"),
            dict(count=7, label="1WEEK", step="day", stepmode="backward"),
            dict(count=1, label="1MNTH", step="month", stepmode="backward"),
            dict(count=6, label="6MNTH", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1YR", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

subfig.update_layout(height=600, width=1200)

# Chart options
subfig.update_layout(plot_bgcolor = "RGB(45,45,48)")  #HEX 2d2d30 
subfig.update_layout(paper_bgcolor = "RGB(37,37,38)") #HEX 252526

subfig.update_layout(
    font_color="RGB(131,148,150)",  	#839496
    title_font_color="RGB(131,148,150)",  	#839496
    legend_title_font_color="RGB(131,148,150)"
    ) #839496

subfig.update_layout(height=600, width=1200)

# Append wq station names to .html files
subfig.write_html("SilicaInConcentrate_ActualValues_ForecastLSTM.html", include_plotlyjs='cdn')
subfig.show()

A comparison of forecasts and actual values of *%silica in concentrate* is displayed above for the period between 1 a.m. August 8, 2017 and 11 p.m. September 9, 2017. The forecasts largely follow the pattern of actual values and contained within the 95% confidence interval.

#### Forecast Comparison with Random Forest
A random forest regressor, in comparison to LSTM, generates greater error and may not perform as well for long-term forecast.

In [ ]:
# Reframe data by lagging features by 1 hour
rf_values = df.drop(['CONC_IRON', 'IRON_FEED_TITLE',
                'SILICA_FEED_TITLE',
                'CONC_IRON_TITLE',
                'CONC_SILICA_TITLE'], axis=1).values
rf_reframed = series_to_supervised(rf_values, n_in=1, n_out=1)

# Define features and target
rf_X = rf_reframed.values[:, :-1]
rf_y = rf_reframed.values[:, -1]

# Split data into train/test sets (80/20)
rf_train_X, rf_test_X, rf_train_y, rf_test_y = train_test_split(
    rf_X, rf_y, test_size=.20, random_state=0, shuffle=False)

# Normalize features
rf_scaler = MinMaxScaler(feature_range=(0,1))
rf_train_X = rf_scaler.fit_transform(rf_train_X)
rf_test_X = rf_scaler.transform(rf_test_X)

# Instantiate regressor
forest = RandomForestRegressor(random_state=1234)

# Fit model on training data
forest.fit(rf_train_X, rf_train_y)

# Make prediction using trained model
rf_yhat = forest.predict(rf_test_X)

# Calculate error scores
rf_mae = mean_absolute_error(rf_test_y, rf_yhat)
rf_rmse = np.sqrt(mean_squared_error(rf_test_y, rf_yhat))
print(32*'-'+'\nFORECAST EVALUATION'+'\n'+32*'-',
      f'\nMean absolute error    : {rf_mae:.4f}',
      f'\nRoot mean squared error: {rf_rmse:.4f}')

In [ ]:
# Plot actual values and forecasts
plt.style.use('ggplot')
fig, ax = plt.subplots(figsize=(18,6))
ax.plot(test_date, rf_test_y, color=palette[1], label='Actual Value')
ax.plot(test_date, rf_yhat, color=palette[2], label='Forecast')
ax.fill_between(test_date, (rf_test_y-ci), (rf_test_y+ci), color=palette[3],
                alpha=.1, label='95% Confidence Interval')
ax.set_title('(%) Silica in Concentrate: Actual Values and Forecasts by Random Forest',
             loc='left', weight='bold', size=16)
ax.set_ylabel('Silica in Concentrate (%)')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.legend()
plt.show()

In [ ]:
from plotly.subplots import make_subplots
subfig = make_subplots(specs=[[{"secondary_y": True}]], shared_yaxes=True)

fig = go.Figure([
    go.Scatter(
        name='(%) Silica in Concentrate: Actual Value',
        x=test_date,
        y=rf_test_y,
        mode='lines',
        line=dict(color='rgb(31, 119, 180)')
    ),
    
    go.Scatter(
        name='(%) Silica in Concentrate: Forecast Value',
        x=test_date,
        y=rf_yhat,
        mode='lines',
        line=dict(color='rgb(62, 100, 240)')
    ),
    
    go.Scatter(
        name='95% CI: Upper Bound',
        x=test_date,
        y=rf_test_y+ci,
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=True
    ),
    
    go.Scatter(
        name='95% CI: Lower Bound',
        x=test_date,
        y=rf_test_y-ci,
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=True
    )
    
])

fig.update_layout(
    yaxis_title='Silica in Concentrate (%)',
    title=' (%) Silica in Concentrate: Actual Values and Forecasts by Random Forest',
    hovermode="x",
    height=500, width=1500
)

subfig.add_traces(fig.data)

subfig.layout.xaxis.title="Date"
subfig.layout.yaxis.title="Silica in Concentrate (%)"
subfig.layout.yaxis.type="linear"

# recoloring is necessary otherwise lines from fig und fig2 would share each color
# e.g. Linear-, Log- = blue; Linear+, Log+ = red... we don't want this
subfig.for_each_trace(lambda t: t.update(line=dict(color=t.marker.color))) 

# Add figure title
subfig.update_layout(title_text = '% Silica in Concentrate: Actual Values and Forecasts by Random Forest: Time Series with Range Slider')

subfig.update_layout(yaxis=dict(title='Silica in Concentrate (%)', tick0=0, anchor='x', rangemode='normal'))
#subfig.update_layout(yaxis2=dict(title='UPPER AND LOWER GUIDELINE RECCOMENDATIONS',
                                # tick0=0, side='right', anchor='x', rangemode='normal'))

# Set y-axes titles
subfig.update_yaxes(title_text="<b>Silica in Concentrate (%)</b>", secondary_y=False, autorange=True)

subfig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=7, label="1WEEK", step="day", stepmode="backward"),
            dict(count=14, label="2WEEK", step="day", stepmode="backward"),
            dict(count=1, label="1MNTH", step="month", stepmode="backward"),
            dict(count=6, label="6MNTH", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1YR", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

subfig.update_layout(height=600, width=1200)

# Chart options
subfig.update_layout(plot_bgcolor = "RGB(45,45,48)")  #HEX 2d2d30 
subfig.update_layout(paper_bgcolor = "RGB(37,37,38)") #HEX 252526

subfig.update_layout(
    font_color="RGB(131,148,150)",  	#839496
    title_font_color="RGB(131,148,150)",  	#839496
    legend_title_font_color="RGB(131,148,150)"
    ) #839496

subfig.update_layout(height=600, width=1200)

# Append wq station names to .html files
subfig.write_html("Silica in Concentrate_ActualValuesForecastRandomForest.html", include_plotlyjs='cdn')
subfig.show()

# Conclusion
A deep learning approach using LSTM was implemented to forecast gangue content in flotation concentrate. Excluding *%iron in concentrate* from the features, *%silica in concentrate* were forecasted one hour ahead and with an error below 1 (based on RMSE, MAE). As the dataset owner stated in [this post](https:\\www.kaggle.com\rogerbellavista\randomforestregressor-mae-0-0922-rmse-0-2314#434654), MAE and RMSE of 1±0.2 is a satisfactory result
for a predictive process. The forecasts are a  promising method for process engineers to assess concentrate purity and take corrective actions in advance, especially when concentrate purity deviates from acceptable values.

This project is an example of a Continuous Improvment (CI) process with potential to optimize mining processes. Having a real time log and backlog
of gangue content can help process engineers and decision makers identify problems in the mining process.

Finally, although LSTM implementation in this notebook has met the objectives, it will benefit from further exploration:
- Forecasting with smaller lag timesteps. For example, a 30-minute lag for the features and inputs.
- Analysis of feature importance in order to understand which parameters of the flotation process greatly affect *%silica in concentrate*. This ensures that the important parameters are adjusted accordingly.